---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [20]:
import pandas as pd
import numpy as np

def blight_model():
    
     
    
    #import train set
    data = pd.read_csv('train.csv', encoding='latin1', low_memory = False)
    #import addresses and merge with data
    adr = pd.read_csv('addresses.csv', encoding = 'latin1', low_memory = False)
    data=pd.merge(data, adr, how="left", left_on = ['ticket_id'], right_on = ['ticket_id'])
    #import lat/lon table and merge with data
    latlon = pd.read_csv('latlons.csv', encoding='latin1', low_memory = False)
    data = pd.merge(data, latlon, how="left", left_on = ['address'], right_on = ['address'])
    
    #deleting non relevant features
    data = data.drop(['payment_amount', 'balance_due',
       'payment_date', 'payment_status', 'collection_status', 'compliance_detail'], axis=1) 
    #deleting rows where compliance is Null
    data = data.dropna(subset=['compliance'])
    
    #   * * * DATA CLEANING - must be done with test data too * * *
    
    data['violation_code_beginwith']=data['violation_code'].apply(lambda x: x.split('-')[0])
    data['violation_code_beginwith']=data['violation_code_beginwith'].apply(lambda x: 'nan' if int(x)>61 else x)
    data['ticket_issued_date'] = pd.to_datetime(data['ticket_issued_date'])
    data['hearing_date'] = pd.to_datetime(data['hearing_date'])
    data['time_passed'] = data['hearing_date']-data['ticket_issued_date']
    data['time_passed'].fillna(0, inplace = True)
    data['time_passed'] = data['time_passed'].apply(lambda x: 0 if x.days < 0 else x.days)
    dummy_violation = pd.get_dummies(data['violation_code_beginwith'], prefix='dummy_violation')
    data = pd.concat([data, dummy_violation], axis=1)
    dummy_agency = pd.get_dummies(data['agency_name'], prefix='dummy_agency')
    data = pd.concat([data, dummy_agency], axis = 1)
    data['lat'].fillna(data['lat'].mean(), inplace=True)
    data['lon'].fillna(data['lon'].mean(), inplace=True)
    
    
    #    * * * MODEL SELECTION* * *
    
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import GradientBoostingClassifier
    
    
    X = data[['fine_amount', 'late_fee', 'discount_amount', 'clean_up_cost',
              'judgment_amount', 'dummy_violation_22', 'time_passed', 'dummy_violation_61', 
               'dummy_violation_9', 'dummy_violation_nan', 'lat', 'lon']]
    #'lat', 'lon',    
    #,          'dummy_agency_Buildings, Safety Engineering & Env Department',
    #           'dummy_agency_Department of Public Works', 'dummy_agency_Detroit Police Department',
    #           'dummy_agency_Health Department', 'dummy_agency_Neighborhood City Halls' ]]
    
    y = data['compliance'].astype(int)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y)
        
    classifier = GradientBoostingClassifier().fit(X, y)

    #a_train = classifier.score(X_train, y_train)
    #a_test = classifier.score(X_test, y_test)
    #print(a_train)
    #print(a_test)
    
    
    
    # * * * import test set, and cleaning the features * * *
    
    test = pd.read_csv('test.csv', encoding='latin1', low_memory = False)
    test = pd.merge(test, adr, how="left", left_on = ['ticket_id'], right_on = ['ticket_id'])
    test = pd.merge(test, latlon, how="left", left_on = ['address'], right_on = ['address'])
    test['violation_code_beginwith']=test['violation_code'].apply(lambda x: x.split('-')[0])
    test['violation_code_beginwith']=test['violation_code_beginwith'].apply(lambda x: 'nan' if int(x)>61 else x)
    test['ticket_issued_date'] = pd.to_datetime(test['ticket_issued_date'])
    test['hearing_date'] = pd.to_datetime(test['hearing_date'])
    test['time_passed'] = test['hearing_date']-test['ticket_issued_date']
    test['time_passed'].fillna(0, inplace = True)
    test['time_passed'] = test['time_passed'].apply(lambda x: 0 if x.days < 0 else x.days)
    dummy_violation_test = pd.get_dummies(test['violation_code_beginwith'], prefix='dummy_violation')
    test = pd.concat([test, dummy_violation_test], axis=1)
    dummy_agency_test = pd.get_dummies(test['agency_name'], prefix='dummy_agency')
    test = pd.concat([test, dummy_agency_test], axis = 1)
    test['lat'].fillna(test['lat'].mean(), inplace=True)
    test['lon'].fillna(test['lon'].mean(), inplace=True)
    
    
    test_for_score = test[['fine_amount', 'late_fee', 'discount_amount', 'clean_up_cost',
              'judgment_amount', 'dummy_violation_22', 'time_passed', 'dummy_violation_61', 
               'dummy_violation_9', 'dummy_violation_nan', 'lat', 'lon']]
    
    test_index = test["ticket_id"]
     
    
    return pd.Series(classifier.predict_proba(test_for_score)[:,1], index = test_index)



In [21]:
blight_model()

ticket_id
284932    0.061538
285362    0.022841
285361    0.072067
285338    0.093617
285346    0.103032
285345    0.093617
285347    0.089908
285342    0.399922
285530    0.033739
284989    0.042243
285344    0.085860
285343    0.038441
285340    0.040352
285341    0.089908
285349    0.103032
285348    0.093617
284991    0.042243
285532    0.044848
285406    0.030823
285001    0.047476
285006    0.042375
285405    0.022841
285337    0.031857
285496    0.067627
285497    0.069891
285378    0.023606
285589    0.032103
285585    0.063944
285501    0.079225
285581    0.023885
            ...   
376367    0.023207
376366    0.036676
376362    0.037724
376363    0.062943
376365    0.023207
376364    0.036676
376228    0.059788
376265    0.035674
376286    0.155734
376320    0.038085
376314    0.034506
376327    0.401464
376385    0.323654
376435    0.423026
376370    0.368751
376434    0.069917
376459    0.075463
376478    0.019080
376473    0.041514
376484    0.034497
376482    0.024013
37